<a href="https://colab.research.google.com/github/jibitesh2004/jibitesh2004/blob/main/Predict_rocket_launch_delays_with_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORT LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import requests  # For API calls

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import pydotplus
from IPython.display import display, clear_output, Image

import ipywidgets as widgets

# For voice output using gTTS
!pip install gTTS
from gtts import gTTS
from IPython.display import Audio

# For reproducibility
np.random.seed(42)


# **Load Historical Excel Data (Past Data)**


In [ ]:
try:
    launch_data = pd.read_excel('Generated.xlsx')
    launch_data.columns = [col.strip() for col in launch_data.columns]  # Remove extra spaces
    print("Excel data loaded successfully.")
except Exception:
    print("Error loading Excel file. Using synthetic historical data for demo.")
    launch_data = pd.DataFrame({
        'Name': ['Rocket A'] * 10,
        'Date': pd.date_range(start='2025-01-01', periods=10),
        'Time (East Coast)': ['10:00 AM'] * 10,
        'Location': ['Cape Canaveral'] * 10,
        'Crewed or Uncrewed': np.random.choice(['Crewed', 'Uncrewed'], 10),
        'Launched?': np.random.choice(['Y', 'N'], 10),
        'Wind Direction': np.random.choice(['N', 'S', 'E', 'W'], 10),
        'Condition': np.random.choice(['Fair', 'Cloudy', 'Stormy'], 10)
    })

# Fill missing values
launch_data.fillna({'Launched?': 'N', 'Crewed or Uncrewed': 'Uncrewed',
                    'Wind Direction': 'unknown', 'Condition': 'Fair'}, inplace=True)

# Synthetic delay times and extra features
launch_data['DelayTime'] = np.random.uniform(10, 60, launch_data.shape[0])
launch_data['Temperature'] = np.random.uniform(15, 35, launch_data.shape[0])  # °C
launch_data['WindSpeed'] = np.random.uniform(0, 50, launch_data.shape[0])       # km/h
launch_data['Precipitation'] = np.random.uniform(0, 5, launch_data.shape[0])      # mm
launch_data['SensorReading'] = np.random.uniform(0.8, 1.0, launch_data.shape[0])  # Simulated sensor
launch_data['CommLog'] = np.random.choice([
    "All systems nominal, ready for launch.",
    "Minor issues detected in fuel sensors, proceed with caution.",
    "Crew on standby, monitoring weather.",
    "Unexpected alert in telemetry data, investigate further."
], launch_data.shape[0])


# **Clean & Process Historical Data**

In [ ]:
# Define target and features
y = launch_data['DelayTime']
cols_to_drop = ['Name', 'Date', 'Time (East Coast)', 'Location', 'Launched?', 'DelayTime']
X = launch_data.drop(columns=cols_to_drop)

# Encode categorical features
label_encoder = LabelEncoder()
for col in ['Crewed or Uncrewed', 'Wind Direction', 'Condition']:
    X[col] = label_encoder.fit_transform(X[col])

numeric_features = ['Temperature', 'WindSpeed', 'Precipitation', 'SensorReading',
                      'Crewed or Uncrewed', 'Wind Direction', 'Condition']
text_feature = 'CommLog'

# Build pipelines for numeric and text data
numeric_pipeline = Pipeline([('scaler', StandardScaler())])
text_pipeline = Pipeline([('tfidf', TfidfVectorizer(max_features=50))])
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('text', text_pipeline, text_feature)
])

# Overall model pipeline
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=0, max_depth=5))
])

# Train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)
model_pipeline.fit(X_train, y_train)
r2_score = model_pipeline.score(X_test, y_test)
print(f"Model R² Score on Historical Test Data: {r2_score:.2f}")


# **Enrich Historical Data with Additional Features**

In [ ]:
def tree_graph_to_png(tree_model, feature_names):
    tree_str = export_graphviz(tree_model,
                               feature_names=feature_names,
                               filled=True,
                               out_file=None)
    graph = pydotplus.graph_from_dot_data(tree_str)
    return Image(graph.create_png())

feature_names = model_pipeline.named_steps['preprocessor'].get_feature_names_out()
decision_tree = model_pipeline.named_steps['regressor']
tree_image = tree_graph_to_png(decision_tree, feature_names=feature_names)
display(tree_image)


# **Define Features & Target for Training**

In [ ]:
def fetch_live_weather(city="Cape Canaveral"):
    """
    Fetch live weather data using the Open-Meteo API.
    Temperature in Celsius.
    """
    latitude = 28.3922
    longitude = -80.6077
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        current_weather = data.get("current_weather", {})
        temperature = current_weather.get("temperature", None)
        wind_speed = current_weather.get("windspeed", None)
        precipitation = 0  # Not provided by current_weather
        return temperature, wind_speed, precipitation
    else:
        return np.random.uniform(15, 35), np.random.uniform(0, 50), np.random.uniform(0, 5)

def simulate_live_space_condition():
    conditions = ["Calm", "Moderate", "Disturbed"]
    return np.random.choice(conditions)

def generate_sensor_alert(sensor_value, threshold=0.9):
    alert_message = ""
    if sensor_value < threshold:
        alert_message += "Alert: Fuel pump sensor reading is below normal. Check required. "
    else:
        alert_message += "Fuel pump sensor operating normally. "
    space_condition = simulate_live_space_condition()
    alert_message += f"Live space condition is {space_condition}."
    return alert_message

def fetch_space_weather(api_key="DEMO_KEY", event_type="FLR"):
    """
    Fetch space weather data from NASA's DONKI API (e.g., solar flares).
    Returns a summary string of the latest event.
    """
    base_url = f"https://api.nasa.gov/DONKI/{event_type}"
    params = {"api_key": api_key}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        events = response.json()
        if events:
            event = events[0]
            event_summary = f"{event_type} on {event.get('beginTime', 'Unknown')}"
        else:
            event_summary = "No significant space weather events."
        return event_summary
    else:
        return "Space weather data unavailable."

# Mappings for categorical features
crew_mapping = {'Crewed': 0, 'Uncrewed': 1}
wind_mapping = {'N': 0, 'S': 1, 'E': 2, 'W': 3, 'unknown': 4}
condition_mapping = {'Fair': 0, 'Cloudy': 1, 'Stormy': 2}


# **Encode Categorical Data for Training**

In [ ]:
# Initialize live data for current prediction
live_temp, live_wind, live_precip = fetch_live_weather()
live_sensor = 0.85  # Simulated sensor reading
live_comm = generate_sensor_alert(live_sensor)
live_space = fetch_space_weather()  # Space weather data

# Prepare sample input (adding 'SpaceWeather' for display only)
sample_input = {
    'Temperature': live_temp,
    'WindSpeed': live_wind,
    'Precipitation': live_precip,
    'SensorReading': live_sensor,
    'Crewed or Uncrewed': crew_mapping['Crewed'],
    'Wind Direction': wind_mapping['N'],
    'Condition': condition_mapping['Fair'],
    'CommLog': live_comm,
    'SpaceWeather': live_space
}
sample_df = pd.DataFrame([sample_input])
print("Live Sample Input for Prediction:")
display(sample_df)

# Predict current delay (model ignores 'SpaceWeather')
predicted_delay_minutes = model_pipeline.predict(sample_df.drop(columns=['SpaceWeather']))[0]
print(f"Predicted Delay Time (Right Now): {predicted_delay_minutes:.1f} minutes")

def generate_delay_statement(input_data, delay_minutes):
    reasons = []
    if input_data['WindSpeed'] > 30:
        reasons.append("high wind speeds")
    if input_data['Precipitation'] > 2:
        reasons.append("heavy precipitation")
    if input_data['SensorReading'] < 0.9:
        reasons.append("suboptimal sensor readings")
    keywords = ['alert', 'check required']
    if any(keyword in input_data['CommLog'].lower() for keyword in keywords):
        reasons.append("live sensor alert indicating potential issues")
    if not reasons:
        reasons.append("all parameters nominal")
    statement = (f"The rocket launch is predicted to be delayed by {delay_minutes:.1f} minutes "
                 f"({delay_minutes*60:.0f} seconds, {delay_minutes/60:.2f} hours) due to "
                 + ", ".join(reasons) + ".")
    return statement

current_explanation = generate_delay_statement(sample_input, predicted_delay_minutes)
print("\nExplanation:")
print(current_explanation)


# **Build a Multimodal Pipeline (Regression)**

In [ ]:
# Widgets for Current Prediction (Right Now)
temp_label = widgets.Label(value=f"Temperature (°C): {live_temp:.1f}")
wind_slider = widgets.FloatSlider(min=0, max=50, step=0.5,
                                  description='Wind Speed (km/h):', value=live_wind)
precip_slider = widgets.FloatSlider(min=0, max=10, step=0.1,
                                    description='Precipitation (mm):', value=live_precip)
sensor_slider = widgets.FloatSlider(min=0.8, max=1.0, step=0.01,
                                    description='Sensor Reading:', value=live_sensor)

# Button for fetching new live data (updates current prediction)
def fetch_live_data(b):
    new_temp, new_wind, new_precip = fetch_live_weather()
    temp_label.value = f"Temperature (°C): {new_temp:.1f}"
    wind_slider.value = new_wind
    precip_slider.value = new_precip
    sensor_slider.value = np.random.uniform(0.8, 1.0)
    current_predict()  # update current prediction

fetch_data_button = widgets.Button(description="Get Right Now Delay")
fetch_data_button.on_click(fetch_live_data)

# Global storage for current prediction data
current_explanation = ""
current_pred = 0.0
current_input_df = None
voice_output = widgets.Output()
message_output = widgets.Output()
data_output = widgets.Output()

def current_predict(change=None):
    global current_explanation, current_pred, current_input_df
    auto_comm = generate_sensor_alert(sensor_slider.value)
    current_input_df = pd.DataFrame({
        'Temperature': [float(temp_label.value.split(":")[1].strip())],
        'WindSpeed': [wind_slider.value],
        'Precipitation': [precip_slider.value],
        'SensorReading': [sensor_slider.value],
        'Crewed or Uncrewed': [crew_mapping['Crewed']],
        'Wind Direction': [wind_mapping['N']],
        'Condition': [condition_mapping['Fair']],
        'CommLog': [auto_comm]
    })
    # Fetch latest space weather for display
    space_weather_status = fetch_space_weather()
    current_input_df["SpaceWeather"] = space_weather_status

    current_pred = model_pipeline.predict(current_input_df.drop(columns=['SpaceWeather']))[0]
    current_explanation = generate_delay_statement(current_input_df.iloc[0].to_dict(), current_pred)

    with voice_output:
        clear_output(wait=True)
        tts = gTTS(text=current_explanation, lang='en')
        tts.save("current_explanation.mp3")
        display(Audio("current_explanation.mp3", autoplay=True))

def show_message(b):
    with message_output:
        clear_output(wait=True)
        print("Prediction:", f"{current_pred:.1f} minutes delay "
              f"({current_pred*60:.0f} seconds, {current_pred/60:.2f} hours)")
        print("Explanation:")
        print(current_explanation)

def view_data(b):
    with data_output:
        clear_output(wait=True)
        # Decode categorical columns for clarity
        temp_df = current_input_df.copy()
        temp_df['Crewed or Uncrewed'] = temp_df['Crewed or Uncrewed'].map({0: 'Crewed', 1: 'Uncrewed'})
        temp_df['Wind Direction'] = temp_df['Wind Direction'].map({0: 'N', 1: 'S', 2: 'E', 3: 'W', 4: 'unknown'})
        temp_df['Condition'] = temp_df['Condition'].map({0: 'Fair', 1: 'Cloudy', 2: 'Stormy'})
        structured_cols = ['Temperature', 'WindSpeed', 'Precipitation', 'SensorReading',
                             'Crewed or Uncrewed', 'Wind Direction', 'Condition']
        unstructured_col = 'CommLog'
        space_col = 'SpaceWeather'
        print("----- Structured Data -----")
        display(temp_df[structured_cols])
        print("\n----- Unstructured Data (CommLog) -----")
        display(temp_df[[unstructured_col]])
        print("\n----- Space Weather -----")
        display(temp_df[[space_col]])
        print("\nPredicted Delay (minutes):", f"{current_pred:.1f}")

# Buttons for current predictions
get_message_button = widgets.Button(description="Get Message")
get_message_button.on_click(show_message)
view_data_button = widgets.Button(description="View Your Data")
view_data_button.on_click(view_data)

# Widgets for Future Prediction
future_day_input = widgets.IntText(value=3, description="Days Ahead:")
future_button = widgets.Button(description="Get Future Delay")

future_output = widgets.Output()

def generate_future_sample():
    # Simulate a future sample with forecast variability
    base_temp, base_wind, base_precip = fetch_live_weather()
    future_temp = np.clip(base_temp + np.random.normal(0, 1), 15, 35)
    future_wind = np.clip(base_wind + np.random.normal(0, 2), 0, 50)
    future_precip = np.clip(base_precip + np.random.normal(0, 0.5), 0, 10)
    future_sensor = np.random.uniform(0.8, 1.0)
    future_comm = generate_sensor_alert(future_sensor)
    future_space = fetch_space_weather()
    return {
        'Temperature': future_temp,
        'WindSpeed': future_wind,
        'Precipitation': future_precip,
        'SensorReading': future_sensor,
        'Crewed or Uncrewed': crew_mapping['Crewed'],
        'Wind Direction': wind_mapping['N'],
        'Condition': condition_mapping['Fair'],
        'CommLog': future_comm,
        'SpaceWeather': future_space
    }

def predict_future_delays(days=3, num_samples=100, delay_threshold=20):
    results = []
    for day in range(1, days+1):
        predictions = []
        for _ in range(num_samples):
            sample = generate_future_sample()
            sample_df = pd.DataFrame([sample])
            pred = model_pipeline.predict(sample_df.drop(columns=['SpaceWeather']))[0]
            predictions.append(pred)
        predictions = np.array(predictions)
        mean_delay = predictions.mean()
        std_delay = predictions.std()
        prob_exceed = np.mean(predictions > delay_threshold) * 100
        results.append({
            'Day': f"Day {day}",
            'Mean Delay (min)': round(mean_delay, 2),
            'Std Dev (min)': round(std_delay, 2),
            f"Prob(Delay > {delay_threshold} min) (%)": round(prob_exceed, 2)
        })
    return pd.DataFrame(results)

def get_future_prediction(b):
    days = future_day_input.value
    future_df = predict_future_delays(days=days, num_samples=100, delay_threshold=20)
    with future_output:
        clear_output(wait=True)
        print(f"Future Delay Predictions for the Next {days} Day(s):")
        display(future_df)

future_button.on_click(get_future_prediction)

# Layout the interactive sections
print("\n--- Current Delay Prediction ---")
display(fetch_data_button, temp_label, wind_slider, precip_slider, sensor_slider)
display(voice_output)
display(get_message_button, message_output)
display(view_data_button, data_output)

print("\n--- Future Delay Prediction ---")
display(future_day_input, future_button, future_output)

# Trigger initial predictions
current_predict()
get_future_prediction(None)


# **Train the Model**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)
model_pipeline.fit(X_train, y_train)
r2_score = model_pipeline.score(X_test, y_test)
print(f"\nModel R² Score on Historical Test Data: {r2_score:.2f}")



# **Visualize the Decision Tree**

In [ ]:
def tree_graph_to_png(tree_model, feature_names):
    tree_str = export_graphviz(tree_model,
                               feature_names=feature_names,
                               filled=True,
                               out_file=None)
    graph = pydotplus.graph_from_dot_data(tree_str)
    return Image(graph.create_png())

feature_names = model_pipeline.named_steps['preprocessor'].get_feature_names_out()
decision_tree = model_pipeline.named_steps['regressor']
tree_image = tree_graph_to_png(decision_tree, feature_names=feature_names)
display(tree_image)

# **Live Data Functions Using Open-Meteo API & Simulated Space Condition**

In [ ]:
def fetch_live_weather(city="Cape Canaveral"):
    """
    Fetch live weather data using the Open-Meteo API.
    For Cape Canaveral, we use its latitude and longitude.
    Open-Meteo is free and requires no API key.
    """
    latitude = 28.3922
    longitude = -80.6077
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        current_weather = data.get("current_weather", {})
        temperature = current_weather.get("temperature", None)  # in Celsius
        if temperature is not None:
            temperature = temperature * 9/5 + 32  # Convert to Fahrenheit
        wind_speed = current_weather.get("windspeed", None)  # in km/h
        if wind_speed is not None:
            wind_speed = wind_speed / 1.609  # Convert to mph
        precipitation = 0  # Not provided by current_weather
        return temperature, wind_speed, precipitation
    else:
        return np.random.uniform(60, 90), np.random.uniform(0, 50), np.random.uniform(0, 5)

def simulate_live_space_condition():
    """
    Simulate live space condition based on an internal algorithm.
    For this demo, we'll simply randomly choose a condition.
    In a real system, this could come from fast internal sensors or anomaly detection.
    """
    conditions = ["Calm", "Moderate", "Disturbed"]
    return np.random.choice(conditions)

def generate_sensor_alert(sensor_value, threshold=0.9):
    """
    Auto-generate an alert based on the sensor reading and live space condition.
    If sensor_value is below threshold, generate an immediate alert.
    Additionally, append the simulated live space condition.
    """
    alert_message = ""
    if sensor_value < threshold:
        alert_message += "Alert: Fuel pump sensor reading is below normal. Check required. "
    else:
        alert_message += "Fuel pump sensor operating normally. "

    space_condition = simulate_live_space_condition()
    alert_message += f"Live space condition is {space_condition}."
    return alert_message


Can you predict whether a launch is likely to happen given specific weather conditions?

# **Sample Live Input Prediction with Live Data, Explanation, & Voice Output**

In [ ]:

# Manual mappings for categorical features.
crew_mapping = {'Crewed': 0, 'Uncrewed': 1}
wind_mapping = {'N': 0, 'S': 1, 'E': 2, 'W': 3, 'unknown': 4}
condition_mapping = {'Fair': 0, 'Cloudy': 1, 'Stormy': 2}

# Fetch live weather data using Open-Meteo API.
live_temp, live_wind, live_precip = fetch_live_weather()
# Simulate a live sensor reading (set below threshold to trigger alert).
live_sensor = 0.85  # below threshold 0.9 triggers alert.
# Auto-generate live alert message using sensor reading and simulated space condition.
live_comm = generate_sensor_alert(live_sensor)

# Prepare a sample live input.
sample_input = {
    'Temperature': live_temp,
    'WindSpeed': live_wind,
    'Precipitation': live_precip,
    'SensorReading': live_sensor,
    'Crewed or Uncrewed': crew_mapping['Crewed'],
    'Wind Direction': wind_mapping['N'],
    'Condition': condition_mapping['Fair'],
    'CommLog': live_comm
}
sample_df = pd.DataFrame([sample_input])
print("\nLive Sample Input for Prediction:")
print(sample_df)

# Predict delay time (in minutes)
predicted_delay_minutes = model_pipeline.predict(sample_df)[0]
predicted_delay_seconds = predicted_delay_minutes * 60
predicted_delay_hours = predicted_delay_minutes / 60
print(f"\nPredicted Delay Time: {predicted_delay_minutes:.1f} minutes "
      f"({predicted_delay_seconds:.0f} seconds, {predicted_delay_hours:.2f} hours)")

def generate_delay_statement(input_data, delay_minutes):
    reasons = []
    if input_data['WindSpeed'] > 30:
        reasons.append("high wind speeds")
    if input_data['Precipitation'] > 2:
        reasons.append("heavy precipitation")
    if input_data['SensorReading'] < 0.9:
        reasons.append("suboptimal sensor readings")
    keywords = ['alert', 'check required']
    if any(keyword in input_data['CommLog'].lower() for keyword in keywords):
        reasons.append("live sensor alert indicating potential issues")
    if not reasons:
        reasons.append("all parameters nominal")
    statement = (f"The rocket launch is predicted to be delayed by {delay_minutes:.1f} minutes "
                 f"({delay_minutes*60:.0f} seconds, {delay_minutes/60:.2f} hours) due to "
                 + ", ".join(reasons) + ".")
    return statement

explanation = generate_delay_statement(sample_input, predicted_delay_minutes)
print("\nExplanation:")
print(explanation)


# **Interactive Demo with Live Data Fetching (No Manual Unstructured Input)**

In [ ]:
print("\nInteractive Demo:")

# Create interactive sliders for live structured data.
temp_slider = widgets.FloatSlider(min=50, max=100, step=0.5, description='Temperature:', value=live_temp)
wind_slider = widgets.FloatSlider(min=0, max=50, step=0.5, description='Wind Speed:', value=live_wind)
precip_slider = widgets.FloatSlider(min=0, max=10, step=0.1, description='Precipitation:', value=live_precip)
sensor_slider = widgets.FloatSlider(min=0.8, max=1.0, step=0.01, description='Sensor Reading:', value=live_sensor)

# Button to fetch live weather data using Open-Meteo API.
def fetch_live_data(b):
    new_temp, new_wind, new_precip = fetch_live_weather()
    temp_slider.value = new_temp
    wind_slider.value = new_wind
    precip_slider.value = new_precip
    sensor_slider.value = np.random.uniform(0.8, 1.0)  # Simulate a new sensor reading.

fetch_live_button = widgets.Button(description="Fetch Live Weather Data")
fetch_live_button.on_click(fetch_live_data)

# Output widget for interactive prediction.
interactive_output = widgets.Output()

def interactive_predict(change=None):
    auto_comm = generate_sensor_alert(sensor_slider.value)
    input_df = pd.DataFrame({
        'Temperature': [temp_slider.value],
        'WindSpeed': [wind_slider.value],
        'Precipitation': [precip_slider.value],
        'SensorReading': [sensor_slider.value],
        'Crewed or Uncrewed': [crew_mapping['Crewed']],
        'Wind Direction': [wind_mapping['N']],
        'Condition': [condition_mapping['Fair']],
        'CommLog': [auto_comm]
    })
    pred = model_pipeline.predict(input_df)[0]
    explanation_text = generate_delay_statement(input_df.iloc[0].to_dict(), pred)
    with interactive_output:
        clear_output(wait=True)
        print(f"Interactive Prediction: {pred:.1f} minutes delay "
              f"({pred*60:.0f} seconds, {pred/60:.2f} hours)")
        print("Explanation:", explanation_text)
        tts_interactive = gTTS(text=explanation_text, lang='en')
        tts_interactive.save("interactive_explanation.mp3")
        display(Audio("interactive_explanation.mp3", autoplay=True))

temp_slider.observe(interactive_predict, names='value')
wind_slider.observe(interactive_predict, names='value')
precip_slider.observe(interactive_predict, names='value')
sensor_slider.observe(interactive_predict, names='value')

display(fetch_live_button, temp_slider, wind_slider, precip_slider, sensor_slider, interactive_output)
interactive_predict()  # Initial prediction